In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/Abstract-generator/bumbleBERT/notebooks

/content/drive/MyDrive/Github/Abstract-generator/bumbleBERT/notebooks


In [3]:
!pip install feedparser tokenizers transformers

     |████████████████████████████████| 81 kB 3.8 MB/s 
     |████████████████████████████████| 3.3 MB 18.1 MB/s 
     |████████████████████████████████| 2.6 MB 37.8 MB/s 
     |████████████████████████████████| 636 kB 58.0 MB/s 
     |████████████████████████████████| 895 kB 62.2 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=753ce7e6b89ebe206ec67e608e1e62864f1cd3f59233f6ee6f125d097dccd9b8
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import os, torch, time, math, sys, re, csv
import numpy as np

sys.path.append('..' + os.sep )
from src import default

from src.data import download as dl, tokenization as tkn, custom_dataset as cd

from torch.utils.data import DataLoader
from src.model.transformer_hf import TransformerModel
from src.model.batching import CustomBatch
from src.model.generate_text import gen_some_text
from src.model.train_evaluate import train, evaluate
#from src.model.transformer import make_gpt_model # imports don't work

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Parameters

In [7]:
maxLen     = 250 # maximum sentence length
bsz        = 3 # batch size
vocabSize  = None # None if you want to let tokenizer do its thing
emsize     = 512 # embedding dimension
nhid       = 2048 # the dimension of the feedforward network model in torch.nn.TransformerEncoder
nlayers    = 12 # the number of torch.nn.TransformerEncoderLayer in torch.nn.TransformerEncoder
nhead      = 8 # the number of heads in the multiheadattention models
dropout    = 0.2 # the dropout value
tknzerType = 'BPE' # type of tokenizing algorithm
trainTokenizer = False # whether to train a new tokenizer or use one already trained
download   = False # haven't implemented yet, whether to download
nbrResults = 10000 # number of data samples to download
epochs = 100 # The number of epochs

### Download Dataset

In [8]:
# download data
filename = f'arxiv_{nbrResults}'
extension = '.csv'
filepath = default.RAW_DATA_DIR + os.sep + filename + extension

print(filepath)

if not os.path.exists(filepath):
    dl.arxiv_api( default.RAW_DATA_DIR, filename, max_results=nbrResults ) # TODO : CHANGE SO THAT NOT CONSTANTLY LOADING DATA
print(f'>> Using {filename} for training <<')

/content/drive/MyDrive/Github/Abstract-generator/bumbleBERT/data/raw/arxiv_10000.csv
>> Using arxiv_10000 for training <<


### Format Dataset

Uses a custom dataset class, which is an iterable and callable structure that returns a sample from our dataset. Within this custom dataset, can determine all preprocessing.

In [9]:
# create dataset
dataset = cd.ArxivDataset(filepath)

### Training Tokenizer

Training of a custom tokenizer. Many options possible here, check the tokenizer training functions to try out various strategies. If he tokenizer for the dataset has already been trained, no need to run this again.

In [ ]:
_ = tkn.train_custom_tokenizer(tknzerType, dataset, filename
                                            , default.TOK_DIR
                                            , vocabSize
                                            , **default.special_token_lst)

### Loading Tokenizer and Splitting Datasets

For some reason, torch tokenizers are not callable as trained. This is confusing, but c'est la vie! Instead, need to load it from file it was saved in using the PreTrainedTokenizerFast class (__call__) implemented in here. Once that's done, you can add this tokenizer as a transform to your dataset! Useful.

We also split the dataset here into training, testing and validation datasets.

In [10]:
tknzrFile = default.TOK_DIR + os.sep + filename + '_' + tknzerType + '.json'

# load PreTrainedTokenizerFast, for __call__. __call__ not implemented in
# the base Tokenizer class... that sounds silly, but it is what it is
tknzr = tkn.load_tokenizer(tknzrFile, **default.special_token_lst)

if vocabSize is None: vocabSize = tknzr.vocab_size

# set tknzr as the transform
dataset.set_transform( tknzr )

# separate dataset into train, test valid TODO : make into a function
fracTrain, fracTest, fracVal = ( 0.7, 0.2, 0.1)
trainTestVal = [ np.floor(fracTrain*len(dataset))\
                    , np.floor(fracTest*len(dataset))\
                    , len(dataset) - ( np.floor( fracTrain*len(dataset) ) +
                    np.floor( fracTest*len(dataset) ) )
                    ]

trainDataset, testDataset, valDataset =\
        torch.utils.data.random_split(dataset, [int(x) for x in trainTestVal]
                                , generator=torch.Generator().manual_seed(42) )


### Creating DataLoaders

Training is done on batches, so we need a way to extract groupings of the data in the appropriate format for our transformer model.
Note that for transformers which we are training, dataloaders outputs both src (x[:-1] and tgt ([1:]).
The collation of batches for different transformer models we have vary. For HuggingFace it's ( maxLen x batch_size ) whereas I think that the Annotated Transformer has ( batch_size x maxLen ).

NOTE : Do not use the tokenizer before the training if you use num_workers>0!
FastTokenizer does not play nicely with forking if you use it before the forking of your data:
https://stackoverflow.com/questions/62691279/how-to-disable-tokenizers-parallelism-true-false-warning

In [11]:
# create dataloaders
# uses collate function to transform batch to correct dimensions
def collate_wrapper(batch):
    return CustomBatch(batch, dim=0, maxLenModel=maxLen, padValue=tknzr.get_vocab()["<pad>"])

# dataloader for training
trainDataLoader = DataLoader(trainDataset, batch_size=bsz, shuffle=True
                                        , num_workers=2
                                        , collate_fn=collate_wrapper
                                        , pin_memory=True
                                        )
# dataloader for validation
valDataLoader = DataLoader(valDataset, batch_size=bsz, shuffle=True
                                        , num_workers=2
                                        , collate_fn=collate_wrapper
                                        , pin_memory=True
                                        )

### Selecting model

Here we choose which model we shall use for training. For now, I've selected the black box Transformer from HuggingFace because the collate_fn I've written gives the correct input size force it... however this can easily be changed! 

In [13]:
# transformer from huggingface
model = TransformerModel(vocabSize, emsize, nhead, nhid, nlayers, dropout).to(device)
#model = TransformerModel(vocabSize, emsize, 10, nhid, nlayers, dropout).to(device)

# transformer from illustrated transformer
#model = make_gpt_model(vocabSize, vocabSize, nlayers, emsize, nhid, nhead, dropout)

criterion = torch.nn.CrossEntropyLoss()
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

### Training

Training loop!

In [20]:
best_val_loss = float("inf")
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train( model, maxLen, trainDataLoader, device, optimizer, scheduler, criterion)
    val_loss = evaluate(model, maxLen, valDataLoader, len(valDataset))
    print('-' * 89)
    print(val_loss)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
                                     # Why is math.exp so large????
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

    
# save best model (two methods)
modelFull = default.MODEL_DIR + os.sep + f'{filename}_epoch{epochs}.pth'
modelWeights = default.MODEL_DIR + os.sep + f'{filename}_weights_epoch{epochs}.pth'
# approach 1: save model (class) entirely (uses pickle)
torch.save(model, modelFull)
# approach 2: save model weights
torch.save(model.state_dict(), modelWeights)

TypeError: ignored

### Text Generation

Here I've simply taken the code Matt uses to generate text.

In [ ]:
# load method full model and just weights
version = f'arxiv_10000_epoch10.pth'
versionWeights = f'arxiv_10000_weights_epoch10.pth'
modelFullPath = default.MODEL_DIR + os.sep + version
modelWeightsPath = default.MODEL_DIR + os.sep + versionWeights

# approach 1: load model (class) entirely (uses pickle)
modelFullLoad = torch.load(modelFullPath, map_location=device)

# approach 2: load model weights, need to have some parameter or something 
modelLoad = TransformerModel(vocabSize, emsize, nhead, nhid, nlayers, dropout).to(device)
modelWeightsLoad = modelLoad.load_state_dict( torch.load(modelWeightsPath) )

RuntimeError: ignored

In [ ]:
# inspect both models
#print('model_A info...\n', modelFullLoad)
#print('\nmodel_B info...\n', modelWeightsLoad)

#print('model_A == model_B:', modelFullLoad == modelWeightsLoad)
#model = modelFullLoad
# Text generation example
prompt = 'Electrons are protons.'
ngen = 60
decode_style = 'sample_full'
model.to('cpu')
generated_text = gen_some_text(
    model, tknzr, 'cpu', maxLen, text_prompt=prompt, tokens_to_gen=ngen, vis=False,
    decode_style=decode_style)
print("Text prompt:\n", prompt)
print("Number of tokens to generate:", ngen)
print("Generated_text:\n", generated_text)

# TODO: alternative generation
# currently 'greedy method'
# see: https://huggingface.co/blog/how-to-generate